In [27]:
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Conv1D, MaxPool1D, MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import joblib
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten, Input, Lambda
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import regularizers
import tensorflow as tf
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import albumentations as A

transform = A.Compose([
    A.RandomBrightnessContrast(),
    A.Affine(scale=[0.8,1.2],translate_percent=0.05, shear=0.2, keep_ratio=True, p=0.5),
    A.Rotate(limit=10)
])

## Without mediapipe

#### Read image and convert it to 28x28 matrix:

In [ ]:
folder_name = '/content/drive/MyDrive/IS4242/data' #might be different for different people
path = folder_name
img_list = []
label_list = []
txt = 'abcdefghiklmnopqrstuvwxy'

for i in tqdm(range(len(os.listdir(path)))):
  filename = os.listdir(path)[i]
  if filename.endswith(('.jpg','.png')):
      label_name = re.split(r'[-_]', filename)[0].lower()
      if label_name not in txt:
        continue
      label_list.append(label_name)
      img = Image.open(os.path.join(path, filename))
      img = img.resize((28, 28), Image.ANTIALIAS)
      img = img.convert('L')
      img_array = np.array(img)
      
      img_list.append(img_array)


In [63]:
X = np.array(img_list)

y = np.array(label_list)
y = np.array(list(map(lambda x:txt.find(x), y)))
y = to_categorical(y)

#### Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

output_dir = '/content/drive/MyDrive/IS4242/dump/'
joblib.dump(X, output_dir+'X.pkl')
joblib.dump(y, output_dir+'y.pkl')

(634, 28, 28)
(159, 28, 28)
(634, 24)
(159, 24)


['/content/drive/MyDrive/IS4242/dump/y.pkl']

#### Augmentation only on train data

In [ ]:
augmented_X = []
augmented_y = []
for i in range(len(X_train)):
  for j in (range(10)):
      transformed = transform(image=X_train[i])
      augmented_X.append(transformed['image'])
      augmented_y.append(y_train[i])

augmented_X = np.array(augmented_X)
augmented_y = np.array(augmented_y)
X_train = np.concatenate([X_train,augmented_X])
y_train = np.concatenate([y_train,augmented_y])

output_dir = '/content/drive/MyDrive/IS4242/dump/'
joblib.dump(augmented_X, output_dir+'augmented_X.pkl')
joblib.dump(augmented_y, output_dir+'augmented_y.pkl')

['/content/drive/MyDrive/IS4242/dump/augmented_y.pkl']

In [ ]:
print(X_train.shape)
print(y_train.shape)

(6974, 28, 28)
(6974, 24)


#### Reshape and divide the pixels by 255

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6974, 28, 28, 1)
(159, 28, 28, 1)
(6974, 24)
(159, 24)


#### CNN with augmentation

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,
        brightness_range=[0.8,1.2],
        vertical_flip=False)


datagen.fit(X_train)

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
# architecture
# Set the CNN model 

model = Sequential()
model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(24, activation = "softmax"))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 conv2d_43 (Conv2D)          (None, 28, 28, 32)        25632     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_21 (Bat  (None, 14, 14, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_28 (Dropout)        (None, 14, 14, 32)        0         
                                                                 
 conv2d_44 (Conv2D)          (None, 14, 14, 64)       

In [ ]:
epochs = 30
batch_size = 32


model_checkpoint_callback = ModelCheckpoint(
    filepath="best.h5",
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max')

es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)

# Fit the model
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test, y_test),steps_per_epoch=X_train.shape[0] // batch_size,
                              verbose = 1,   callbacks=[model_checkpoint_callback,learning_rate_reduction,es])

Epoch 1/30


<ipython-input-169-021abc8791b3>:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),


54/54 [==============================] - ETA: 0s - loss: 3.4244 - accuracy: 0.0540
Epoch 1: val_accuracy improved from -inf to 0.01887, saving model to best.h5
54/54 [==============================] - 81s 1s/step - loss: 3.4244 - accuracy: 0.0540 - val_loss: 3.2231 - val_accuracy: 0.0189 - lr: 0.0010
Epoch 2/30
54/54 [==============================] - ETA: 0s - loss: 3.1336 - accuracy: 0.0711
Epoch 2: val_accuracy improved from 0.01887 to 0.06289, saving model to best.h5
54/54 [==============================] - 82s 2s/step - loss: 3.1336 - accuracy: 0.0711 - val_loss: 3.2113 - val_accuracy: 0.0629 - lr: 0.0010
Epoch 3/30
54/54 [==============================] - ETA: 0s - loss: 3.0425 - accuracy: 0.0963
Epoch 3: val_accuracy did not improve from 0.06289
54/54 [==============================] - 77s 1s/step - loss: 3.0425 - accuracy: 0.0963 - val_loss: 3.2515 - val_accuracy: 0.0440 - lr: 0.0010
Epoch 4/30
54/54 [==============================] - ETA: 0s - loss: 2.8801 - accuracy: 0.1382
E

In [ ]:
results = model.predict(X_test)
y_pred = np.argmax(results,axis = 1) 
y_true = np.argmax(y_test,axis = 1) 


5/5 [==============================] - 0s 71ms/step


#### Results

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.25      0.40      0.31         5
           2       0.83      0.42      0.56        12
           3       0.50      0.40      0.44         5
           4       0.38      0.43      0.40         7
           5       0.57      0.67      0.62         6
           6       0.00      0.00      0.00         5
           7       0.25      0.09      0.13        11
           8       0.75      0.60      0.67         5
           9       0.75      0.50      0.60         6
          10       0.67      0.67      0.67         3
          11       0.00      0.00      0.00         3
          12       0.11      0.20      0.14         5
          13       0.57      0.50      0.53         8
          14       0.67      0.57      0.62         7
          15       0.33      0.17      0.22         6
          16       0.29      0.25      0.27         8
          17       0.50    

#### Prediction and evaluation

In [ ]:
img = Image.open('m.png')
img = img.resize((28, 28), Image.ANTIALIAS)
img = img.convert('L')
img_array = np.array(img)
img_array = img_array/255.0
img_array = img_array.reshape(-1, 28, 28, 1)

In [ ]:
results = model.predict(img_array)
y_pred = np.argmax(results,axis = 1)
txt[y_pred[0]]

1/1 [==============================] - 0s 46ms/step


'm'

#### After hyperparameter tuning, train it with the whole data

In [3]:
output_dir = '/content/drive/MyDrive/IS4242/dump/'
X = joblib.load(output_dir+'X.pkl')
y = joblib.load(output_dir+'y.pkl')

In [66]:
augmentX = []
augmenty = []
for i in range(len(X)):
  for j in (range(10)):
      transformed = transform(image=X[i])
      augmentX.append(transformed['image'])
      augmenty.append(y[i])

augmentX = np.array(augmentX)
augmenty = np.array(augmenty)
X = np.concatenate([X,augmentX])
y = np.concatenate([y,augmenty])

In [8]:
X = X/255.0
X = X.reshape(-1, 28, 28, 1)

In [9]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,
        brightness_range=[0.8,1.2],
        vertical_flip=False)


datagen.fit(X)

In [10]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2) , padding = 'same'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2) , padding = 'same'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , padding = 'same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(512, activation = "relu"))
model.add(Dense(24, activation = "softmax"))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batch_size = 64

# Fit the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(datagen.flow(X, y, batch_size=batch_size),epochs = epochs)

In [ ]:
img = Image.open('m.png')
img = img.resize((28, 28), Image.ANTIALIAS)
img = img.convert('L')
img_array = np.array(img)
img_array = img_array/255.0
img_array = img_array.reshape(-1, 28, 28, 1)

In [ ]:
results = model.predict(img_array)
y_pred = np.argmax(results,axis = 1)
txt[y_pred[0]]

1/1 [==============================] - 0s 46ms/step


'm'

## Using mediapipe

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 40.8 MB/s eta 0:00:00


In [2]:
# For capturing hand coordinates
import cv2
import tensorflow as tf
import mediapipe as mp

In [3]:
# For processing data
import csv
import os
import numpy as np
import pandas as pd
from tqdm import tqdm 
import re

#### Convert image data to matrix

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import albumentations as A

transform = A.Compose([
    A.RandomBrightnessContrast(),
    A.Affine(scale=[0.8,1.2],translate_percent=0.05, shear=0.2, keep_ratio=True, p=0.5),
    A.Rotate(limit=10)
])

In [ ]:
folder_name = '/content/drive/MyDrive/IS4242/data' #might be different for different people
path = folder_name
img_list = []
label_list = []
txt = 'abcdefghiklmnopqrstuvwxy'

for i in tqdm(range(len(os.listdir(path)))):
  filename = os.listdir(path)[i]
  if filename.endswith(('.jpg','.png')):
      label_name = re.split(r'[-_]', filename)[0].lower()
      if label_name not in txt:
        continue
      label_list.append(label_name)
      img = Image.open(os.path.join(path, filename))
      img = img.resize((224, 224), Image.ANTIALIAS) # bigger size
      img = img.convert('L')
      img_array = np.array(img)
      
      img_list.append(img_array)

X = np.array(img_list)

y = np.array(label_list)
y = np.array(list(map(lambda x:txt.find(x), y)))
y = to_categorical(y)

output_dir = '/content/drive/MyDrive/IS4242/dump/'
joblib.dump(X,output_dir+'X_224.pkl')
joblib.dump(y,output_dir+'y_224.pkl')


 16%|█▌        | 196/1221 [00:26<03:02,  5.63it/s]

In [109]:
output_dir = '/content/drive/MyDrive/IS4242/dump/'
X = joblib.load(output_dir+'X_224.pkl')
y = joblib.load(output_dir+'y_224.pkl')

In [110]:
len(X)

817

#### Train test split

In [111]:
from sklearn.model_selection import train_test_split

# We will take 33% from 1000 for our test data.
# Recommended value 80:20, 67:33, 50:50
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Augmentation only on train data

In [112]:
augmented_X = []
augmented_y = []
for i in range(len(X_train)):
  for j in (range(10)):
      transformed = transform(image=X_train[i])
      augmented_X.append(transformed['image'])
      augmented_y.append(y_train[i])

augmented_X = np.array(augmented_X)
augmented_y = np.array(augmented_y)
X_train = np.concatenate([X_train,augmented_X])
y_train = np.concatenate([y_train,augmented_y])

#### Generate dataset of hand points

In [113]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [116]:
data = []
labels = []

In [128]:
count = 0
with mp_hands.Hands(static_image_mode =True, max_num_hands = 2, min_detection_confidence=0.5) as hands:
    for i in tqdm(range(len(X_train))):
        # Load the image and extract its features
            results = hands.process(cv2.cvtColor(cv2.flip(X_train[i],1), cv2.COLOR_BGR2RGB))
            try:
                # Extract Hand landmarks
                for hand_landmark in results.multi_hand_landmarks:
                    right_hand = hand_landmark.landmark
                right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())
                # Concate rows
                row = right_hand_row

                # Extract the label from the image filename (e.g. "A.jpg")
                label = y_train[i]

                data.append(row)
                labels.append(label)
                print(row, label)
            except Exception as e:
                count += 1
                continue


Output hidden; open in https://colab.research.google.com to view.

In [129]:
count

4706

In [130]:
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.570230,0.700385,0.460918,0.601834,0.385230,0.526549,0.322623,0.470859,0.301588,0.408549,...,0.434779,0.504095,0.549578,0.469310,0.416643,0.467967,0.423916,0.518278,0.466331,0.533848
1,0.446183,0.796893,0.341452,0.681229,0.304357,0.500339,0.314089,0.366598,0.315794,0.253817,...,0.488412,0.601652,0.734732,0.564161,0.699286,0.469736,0.620621,0.544259,0.574593,0.615079
2,0.554823,0.467337,0.477951,0.500206,0.370337,0.497704,0.303738,0.458727,0.319072,0.396847,...,0.466813,0.410920,0.478718,0.277710,0.441688,0.218711,0.420079,0.179862,0.393535,0.143242
3,0.421717,0.593616,0.332333,0.543942,0.278878,0.457589,0.282298,0.384428,0.314716,0.334537,...,0.391403,0.521059,0.528068,0.395230,0.475645,0.423229,0.459146,0.468935,0.459199,0.488524
4,0.651359,0.619728,0.656911,0.514619,0.602316,0.457691,0.539696,0.429281,0.491382,0.435781,...,0.391995,0.521948,0.437633,0.637292,0.367256,0.589515,0.384333,0.575371,0.412115,0.575475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156,0.583231,0.452619,0.539714,0.455225,0.495395,0.492904,0.479851,0.551855,0.483757,0.602373,...,0.545168,0.576425,0.487555,0.524171,0.515908,0.578480,0.545968,0.580548,0.566784,0.575146
3157,0.562530,0.498690,0.511731,0.506001,0.454501,0.544319,0.426224,0.600555,0.422746,0.650416,...,0.465353,0.691858,0.439111,0.577353,0.453883,0.642066,0.476830,0.662664,0.497206,0.675142
3158,0.563214,0.502042,0.515475,0.514002,0.460635,0.550754,0.430844,0.599683,0.423958,0.642966,...,0.467354,0.680093,0.444119,0.574531,0.456100,0.633061,0.477790,0.649635,0.497778,0.658029
3159,0.583231,0.452619,0.539714,0.455225,0.495395,0.492904,0.479851,0.551855,0.483757,0.602373,...,0.545168,0.576425,0.487555,0.524171,0.515908,0.578480,0.545968,0.580548,0.566784,0.575146


In [140]:
import joblib

output_dir = f"/content/drive/MyDrive/IS4242/dump/"
joblib.dump(data, output_dir+'data_cnn_train.pkl')
joblib.dump(labels, output_dir+'labels_cnn_train.pkl')

['/content/drive/MyDrive/IS4242/dump/labels_cnn_train.pkl']

In [7]:
# import joblib
# output_dir = f"/content/drive/MyDrive/IS4242/dump/"
# data = joblib.load(output_dir+'data_knn_2.pkl')
# labels = joblib.load(output_dir+'labels_knn_2.pkl')

In [134]:
labels.shape

(3161, 24)

In [84]:
len(X_train[0])

42

In [ ]:
test_data = []
test_label = []
count = 0
with mp_hands.Hands(static_image_mode =True, max_num_hands = 2, min_detection_confidence=0.5) as hands:
    for i in tqdm(range(len(X_test))):
        # Load the image and extract its features
            results = hands.process(cv2.cvtColor(cv2.flip(X_test[i],1), cv2.COLOR_BGR2RGB))
            try:
                # Extract Hand landmarks
                for hand_landmark in results.multi_hand_landmarks:
                    right_hand = hand_landmark.landmark
                right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())
                # Concate rows
                row = right_hand_row

                # Extract the label from the image filename (e.g. "A.jpg")
                label = y_test[i]

                test_data.append(row)
                test_label.append(label)
                print(row, label)
            except Exception as e:
                count += 1
                continue

In [141]:
labels = np.array(labels)
test_label = np.array(test_label)

In [137]:
x_train = np.array(data)
x_test = np.array(test_data)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
print(x_train.shape)
print(x_test.shape)

(3161, 42, 1)
(44, 42, 1)


In [96]:
y_test.shape

(636, 24)

#### CNN Architecture with augmentation

In [138]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [139]:
# architecture
# Set the CNN model 

# One Dimensional Convolutional Neural Network model, Train will be feed to 1 Dimension Convolutional Neural Network
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=x_train.shape[1:3]),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(rate=0.2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(24, activation='softmax')])

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_25 (Conv1D)          (None, 42, 32)            192       
                                                                 
 conv1d_26 (Conv1D)          (None, 42, 32)            5152      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 21, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_27 (Conv1D)          (None, 21, 64)            10304     
                                                                 
 conv1d_28 (Conv1D)          (None, 21, 64)            20544     
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 10, 64)           0         
 g1D)                                                 

In [143]:
epochs = 30
batch_size = 32


model_checkpoint_callback = ModelCheckpoint(
    filepath="best.h5",
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max')

es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)

# Fit the model
history = model.fit(x_train, labels, batch_size=batch_size,
                              epochs = epochs, validation_data = (x_test, test_label),steps_per_epoch=x_train.shape[0] // batch_size,
                              verbose = 1,   callbacks=[model_checkpoint_callback,learning_rate_reduction,es])

Epoch 1/30
95/98 [============================>.] - ETA: 0s - loss: 3.1663 - accuracy: 0.0507
Epoch 1: val_accuracy improved from -inf to 0.09091, saving model to best.h5
98/98 [==============================] - 7s 12ms/step - loss: 3.1661 - accuracy: 0.0510 - val_loss: 3.1543 - val_accuracy: 0.0909 - lr: 0.0010
Epoch 2/30
98/98 [==============================] - ETA: 0s - loss: 3.0890 - accuracy: 0.0751
Epoch 2: val_accuracy improved from 0.09091 to 0.11364, saving model to best.h5
98/98 [==============================] - 1s 10ms/step - loss: 3.0890 - accuracy: 0.0751 - val_loss: 2.9516 - val_accuracy: 0.1136 - lr: 0.0010
Epoch 3/30
94/98 [===========================>..] - ETA: 0s - loss: 2.9562 - accuracy: 0.0963
Epoch 3: val_accuracy improved from 0.11364 to 0.18182, saving model to best.h5
98/98 [==============================] - 1s 9ms/step - loss: 2.9550 - accuracy: 0.0952 - val_loss: 2.7136 - val_accuracy: 0.1818 - lr: 0.0010
Epoch 4/30
95/98 [============================>.] - E

#### Prediction and evaluation

In [144]:
results = model.predict(x_test)
y_pred = np.argmax(results,axis = 1) 
y_true = np.argmax(test_label,axis = 1) 


2/2 [==============================] - 0s 5ms/step


In [145]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       0.00      0.00      0.00         1
           2       0.67      1.00      0.80         2
           3       0.00      0.00      0.00         1
           4       0.50      0.50      0.50         2
           5       0.00      0.00      0.00         0
           6       1.00      1.00      1.00         1
           7       0.67      1.00      0.80         2
           8       0.50      0.50      0.50         2
           9       0.00      0.00      0.00         1
          10       1.00      0.67      0.80         3
          11       0.00      0.00      0.00         1
          12       0.25      0.50      0.33         2
          13       1.00      0.50      0.67         2
          14       1.00      1.00      1.00         3
          16       0.75      1.00      0.86         3
          17       0.00      0.00      0.00         1
          18       1.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

In [146]:
from sklearn.metrics import roc_auc_score
import numpy as np

# Calculate the ROC AUC using the micro-averaging method
roc_auc = roc_auc_score(test_label, results, multi_class='ovo', average='micro')
print("ROC AUC score:", roc_auc)


ROC AUC score: 0.9500089831117499
